This is a copy of with minor adjustments
https://colab.research.google.com/drive/1a_GgkJ-nirZ3hYk2fsYmPmEsGQcGMuXC?usp=sharing#scrollTo=4FSdFfE35HGs

In [5]:
import pandas as pd
import sklearn.metrics
import numpy as np
import random

In [2]:
PATH = "./dunnhumby/"
OUTPUT_PATH = PATH +'output/'
# set PATH for Dunnhumby Journey dataset
DUNNHUMBY_JOURNEY_PATH = PATH + "dunnhumby_The-Complete-Journey/"


In [3]:
!wget https://www.dunnhumby.com/wp-content/uploads/sourcefiles/dunnhumby_The-Complete-Journey.zip -P $PATH

--2021-09-20 20:05:12--  https://www.dunnhumby.com/wp-content/uploads/sourcefiles/dunnhumby_The-Complete-Journey.zip
Resolving www.dunnhumby.com (www.dunnhumby.com)... 2620:1ec:29::64, 2620:1ec:48::64, 13.107.213.64, ...
Connecting to www.dunnhumby.com (www.dunnhumby.com)|2620:1ec:29::64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307038175 (293M) [application/zip]
Saving to: ‘./dunnhumby/dunnhumby_The-Complete-Journey.zip’

dunnhumby_The-Compl 100%[===================>] 292.81M  3.89MB/s    in 66s     

2021-09-20 20:06:18 (4.43 MB/s) - ‘./dunnhumby/dunnhumby_The-Complete-Journey.zip’ saved [307038175/307038175]



In [4]:
!unzip -d $PATH -o -q $PATH/dunnhumby_The-Complete-Journey.zip

In [6]:
df = pd.read_csv(DUNNHUMBY_JOURNEY_PATH+"dunnhumby - The Complete Journey CSV/transaction_data.csv", delimiter = ",")
products = pd.read_csv(DUNNHUMBY_JOURNEY_PATH+"dunnhumby - The Complete Journey CSV/product.csv", delimiter = ",", index_col=0)

In [7]:
df.head()


,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [8]:
df_grouped = df.groupby('BASKET_ID')

# Clique expansion

The dunnhumby_cleora_input.txt file has the structure of adjacency list.

In [10]:
df_grouped.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2595723,1598,42305362535,711,45277,1,1.00,3228,-0.50,1520,102,0.0,0.0
2595724,1598,42305362535,711,46427,1,1.16,3228,0.00,1520,102,0.0,0.0
2595725,1598,42305362535,711,50293,1,0.55,3228,0.00,1520,102,0.0,0.0
2595726,1598,42305362535,711,52641,1,1.25,3228,-0.35,1520,102,0.0,0.0


In [11]:
cleora_f_dict = open(PATH+'dunnhumby_cleora_input.txt', 'w') 
for name, group in df_grouped:
    product_ids = [str(p) for p in group['PRODUCT_ID'].tolist()]
    cleora_f_dict.write('{}\n'.format(' '.join(product_ids)))
cleora_f_dict.close()

**Setting configuration before Train Cleora.**

1.   cleora_n_iter 
2.   cleora_dim
3.   columns

and other if you want. More about configuration params you can find [here.](https://cleora.readthedocs.io/en/latest/running.html#run-options)


In [12]:
config = {
    #embedding computation
    'cleora_n_iter': 4,
    'cleora_dim': 1024,
    'columns' : 'complex::reflexive::CliqueNode'
}

The parameter -c "**complex::reflexive::CliqueNode**" means that edges will be 

created for all cominations of nodes from each line. 
This translates to clique expansion scheme.

*    You can read more about **Clique Node** and other expansion methods you can read [here.](https://cleora.readthedocs.io/en/latest/algorithms.html#clique-expansion)


In [13]:
import subprocess

def train_cleora(dim, n_iter, columns, input_filename):
    command = ['./target/release/cleora',
                '--columns', columns,
                '--dimension', str(dim), 
                '-n', str(n_iter), 
                '--input', input_filename,
                '-o',OUTPUT_PATH,
               '-r', f'iterations_{n_iter}_'
               ]
    subprocess.run(command, check=True)

In [14]:
%%time
train_cleora(config['cleora_dim'], config['cleora_n_iter'], config['columns'], PATH+'dunnhumby_cleora_input.txt')

CPU times: user 6.34 ms, sys: 16.6 ms, total: 22.9 ms
Wall time: 1min 9s


# Prediction

In [15]:
prods_for_ar = []

for name, group in df_grouped:
    product_ids = [str(p) for p in group['PRODUCT_ID'].tolist()]
    prods_for_ar.append(product_ids)

In [17]:
!ls $OUTPUT_PATH

iterations_4___CliqueNode__CliqueNode.out


In [20]:
vects_iter = np.loadtxt(OUTPUT_PATH+f"iterations_{config['cleora_n_iter']}___CliqueNode__CliqueNode.out", delimiter = " ", skiprows=1, usecols=[0]+list(range(2,1026)))

In [21]:
ids_iter = vects_iter[:, 0]
vects_iter = vects_iter[:, 1:]

In [22]:
dist = sklearn.metrics.pairwise.cosine_similarity(vects_iter[0].reshape(1, -1), vects_iter, dense_output=True)
ranking = (-dist).argsort()

In [ ]:
for idx, v in enumerate(vects_iter):
    print('-------------base product:')
    curr_prod_id = ids_iter[idx]
    print(curr_prod_id)
    curr_product = products.loc[curr_prod_id]

    print(curr_product['COMMODITY_DESC'], curr_product['SUB_COMMODITY_DESC'], curr_product['CURR_SIZE_OF_PRODUCT'])
    dist = sklearn.metrics.pairwise.cosine_similarity(v.reshape(1, -1), vects_iter, dense_output=True)
    ranking = (-dist).argsort()[0]
    for r in ranking[:10]:
        suggested_prod_id = int(ids_iter[r])
        suggested_prod = products.loc[suggested_prod_id]
        print('suggested: ', suggested_prod_id, suggested_prod['COMMODITY_DESC'], suggested_prod['SUB_COMMODITY_DESC'], suggested_prod['CURR_SIZE_OF_PRODUCT'])

-------------base product:
1004906.0
POTATOES POTATOES RUSSET (BULK&BAG) 5 LB
suggested:  1004906 POTATOES POTATOES RUSSET (BULK&BAG) 5 LB
suggested:  15483424 CRACKERS/MISC BKD FD BUTTER SPRAY CRACKER (RITZ/CLU 15.5 OZ
suggested:  908105 SOUP BOUILLON N   15 CT
suggested:  1138220 SPICES & EXTRACTS SPICES & SEASONINGS 2.50 OZ
suggested:  1076608 COLD CEREAL ADULT CEREAL 17.3 OZ
suggested:  872117 PORK LOIN - WH/HALF BONELESS  
suggested:  8157372 FRZN MEAT/MEAT DINNERS FROZEN PASTA  
suggested:  8157388 FRZN MEAT/MEAT DINNERS FROZEN PASTA  
suggested:  15972701 CHRISTMAS  SEASONAL BAGS/TAGS/TISSUE/BOXES  
suggested:  9677065 MAKEUP AND TREATMENT SEASONAL PRESHCOOL  
-------------base product:
1033142.0
ONIONS ONIONS SWEET (BULK&BAG) 40 LB
suggested:  1033142 ONIONS ONIONS SWEET (BULK&BAG) 40 LB
suggested:  916122 CHICKEN CHICKEN BREAST BONELESS  
suggested:  9297556 DRY BN/VEG/POTATO/RICE MISC GRAIN MIXES 6.6 OZ
suggested:  1069094 DRY NOODLES/PASTA NOODLES DRY 1 LB
suggested:  860501

suggested:  920955 BEEF GRND/PATTY - ROUND  
suggested:  17973385 REFRIGERATED YOGURT 6 OZ
suggested:  1026820 DOMESTIC WINE POPULAR 750ML WINES 750 ML
suggested:  7145621 MAKEUP AND TREATMENT COVERGIRL  
suggested:  1039460 REFRGRATD DOUGH PRODUCTS MISC REFRIG DOUGH PRODUCTS 11.4 OZ
suggested:  977220 COOKIES/CONES GRAHAM CRACKERS 14.4 OZ
suggested:  882974 PNT BTR/JELLY/JAMS PRESERVES JAM MARMALADE 10 OZ
suggested:  16731400 BOOKSTORE HARDBACK/TRADE EVERYDAY  
suggested:  7157249 MAKEUP AND TREATMENT COVERGIRL  
suggested:  890273 SNKS/CKYS/CRKR/CNDY COOKIES/SWEET GOODS 6.25 OZ
-------------base product:
937406.0
BREAKFAST SAUSAGE/SANDWICHES LINKS - RAW 12OZ
suggested:  937406 BREAKFAST SAUSAGE/SANDWICHES LINKS - RAW 12OZ
suggested:  1083150 CEREAL/BREAKFAST CEREAL - COLD 10.6 OZ
suggested:  884122 CANDY - PACKAGED PROCESSED OTHER 8 OZ
suggested:  984411 DRY BN/VEG/POTATO/RICE RICE SIDE DISH MIXES DRY 10 OZ
suggested:  1035258 POPCORN POPCORN - MICROWAVE 35 OZ
suggested:  916584 MILK

suggested:  985911 BAKED SWEET GOODS SWEET GOODS - FULL SIZE 14 OZ
suggested:  956737 DRY SAUCES/GRAVY SAUCE MIXES GRAVY MIXES DRY 1.31 OZ
suggested:  1106340 PICKLE/RELISH/PKLD VEG RELISHES B  32 OZ
suggested:  981427 PORK EXTERNAL FRESH  
suggested:  13511309 ORAL HYGIENE PRODUCTS MOUTHWASH RINSES AND SPRAYS  
suggested:  6919597 DRY BN/VEG/POTATO/RICE RICE SIDE DISH MIXES DRY 5.8 OZ
suggested:  7409790 MEAT - SHELF STABLE CHILI: CANNED 15 OZ
suggested:  8066815 MAGAZINE CROSSWORDS/PUZZLES-MAGAZINE  
suggested:  847232 SOUP DRY SOUP 4.2 OZ/2CT
suggested:  943241 SEAFOOD - FROZEN FRZN GRILLED PORTIONS 7.6 OZ
-------------base product:
1087347.0
BAG SNACKS SGL SV/VEND MACH SNACKS CHIP/P .875 OZ
suggested:  1087347 BAG SNACKS SGL SV/VEND MACH SNACKS CHIP/P .875 OZ
suggested:  902451 CANDY - PACKAGED CANDY BAGS-CHOCOCLATE 1.25 OZ
suggested:  15595999 FIRST AID PRODUCTS ELSTC BNDGES SUPRT HOSE  
suggested:  943905 MISC WINE BEERALEMALT LIQUORS 12OZ/4PK
suggested:  1128101 SPRING/SUMMER SE

suggested:  1106523 FLUID MILK PRODUCTS FLUID MILK WHITE ONLY 1 GA
suggested:  1121699 CANNED JUICES APPLE JUICE & CIDER (OVER 50% 11.5 OZ
suggested:  1012530 HAIR CARE PRODUCTS DANDRUFF-MEDICATED SHAMPOOS 13.5 OZ
suggested:  9832275 CRACKERS/MISC BKD FD MULTI-PACK CRACKERS  
suggested:  1048299 CANNED JUICES VEG JUICE (EXCEPT TOMATO) (OVE 46 OZ
suggested:  968668 FRZN JCE CONC/DRNKS FRZN CONC UNDER 50% JUICE 11.5 OZ
suggested:  1066959 DRY BN/VEG/POTATO/RICE DRY BEANS/PEAS/BARLEY: BAG & B 16 OZ
suggested:  858553 FRZN JCE CONC/DRNKS FRZN CONC ALLIEDS OVER 50% JUI 11.5 OZ
suggested:  946900 FRZN JCE CONC/DRNKS FRZN CONC ALLIEDS OVER 50% JUI 11.5 OZ
suggested:  1015380 PICKLE/RELISH/PKLD VEG PICKLES 46 OZ
-------------base product:
1107553.0
SOFT DRINKS SFT DRNK SNGL SRV BTL CARB (EX 20 OZ
suggested:  1107553 SOFT DRINKS SFT DRNK SNGL SRV BTL CARB (EX 20 OZ
suggested:  6552015 WAREHOUSE SNACKS CANISTER POTATO/TORT CHIPS 2.82 OZ
suggested:  15928012 DIETARY AID PRODUCTS DIET CNTRL BARS N

suggested:  854852 TOMATOES TOMATOES HOTHOUSE ON THE VINE 13 LB
suggested:  904360 VEGETABLES SALAD HEAD LETTUCE 24CT
suggested:  1100338 DINNER MXS:DRY MEAT INCLUDED 8.5 OZ
suggested:  927848 BATH BATH BUBBLES-BATH SOAPS-ADDITI 32 OZ
suggested:  877134 HOUSEHOLD CLEANG NEEDS TOILET BOWL MANUAL 28 CT
suggested:  1046521 FRUIT - SHELF STABLE FRUIT COCKTAIL FRUIT SALAD 29 OZ
suggested:  1124352 HEAT/SERVE CHICKEN-FULLY COOKED 6 OZ
suggested:  997450 ICE CREAM/MILK/SHERBTS PREMIUM 56 OZ
suggested:  878880 YOGURT YOGURT NOT MULTI-PACKS 6 OZ
suggested:  929326 HOSIERY/SOCKS LEGGS  
-------------base product:
865528.0
BACON PREMIUM 16OZ
suggested:  865528 BACON PREMIUM 16OZ
suggested:  821562 DRY BN/VEG/POTATO/RICE NOODLE SIDE DISH MIXES 4.8 OZ
suggested:  928754 CHEESE CREAM CHEESE 8 OZ
suggested:  1138816 PICKLE/RELISH/PKLD VEG PICKLES 16 OZ
suggested:  13416297 MAKEUP AND TREATMENT REVLON  
suggested:  825006 APPLES APPLES RED DELICIOUS (BULK&BAG 3 LB
suggested:  912902 PASTA SAUCE MAINST

...

suggested:  839418 KITCHEN GADGETS GADGETS/TOOLS  
suggested:  1111883 HAIR CARE PRODUCTS HAIR CONDITIONERS AND RINSES 13.5 OZ
suggested:  1015306 DOMESTIC WINE FIGHTING VARIETAL WINES 1.5 LTR
suggested:  823110 DRY SAUCES/GRAVY SAUCE MIXES GRAVY MIXES DRY 1 7/8 OZ
suggested:  992817 CANNED JUICES BLENDED JUICE&COMBINATIONS (OV 64 OZ
suggested:  825437 PICKLE/RELISH/PKLD VEG PICKLES 24 OZ
suggested:  995242 FLUID MILK PRODUCTS FLUID MILK WHITE ONLY  
suggested:  899690 COFFEE FILTERS BASIC OPENSTOCK  
suggested:  6391122 CONDIMENTS/SAUCES HOT SAUCE 12 OZ
suggested:  14043774 IN-STORE PHOTOFINISHING ONE HOUR PROCESSING  
-------------base product:
921449.0
FLORAL-FLOWERING PLANTS PREMIUM FLOWERING PLANTS 6 IN
suggested:  1138829 STATIONERY & SCHOOL SUPPLIES CD-R 5 CT
suggested:  1121434 REFRGRATD DOUGH PRODUCTS REFRIGERATED BISCUITS REGULAR 12 OZ
suggested:  921449 FLORAL-FLOWERING PLANTS PREMIUM FLOWERING PLANTS 6 IN
suggested:  1133160 STATIONERY & SCHOOL SUPPLIES SCHOOL PAPER 500 CT


suggested:  5565026 SHAVING CARE PRODUCTS RAZORS AND BLADES  
suggested:  12673429 TOYS AND GAMES SEASONAL PRESHCOOL  
suggested:  8069382 MAGAZINE CROSSWORDS/PUZZLES-MAGAZINE  
suggested:  1229538 COOKIES/CONES SANDWICH COOKIES  
suggested:  8091665 GREETING CARDS/WRAP/PARTY SPLY CARDS EVERYDAY  
suggested:  12782129 BAG SNACKS SGL SV/VEND MACH SNACKS CHIP/P 2.125 OZ
suggested:  1132246 FLUID MILK PRODUCTS CHOCOLATE MILK 1 GA
suggested:  6548473 FOOT CARE PRODUCTS FOOT CARE - MEDICATED CORN/CAL  
suggested:  905306 BAKED SWEET GOODS PIES: FRUIT/NUT 4 OZ
suggested:  480014 COUPON/MISC ITEMS GASOLINE-REG UNLEADED  
-------------base product:
9337841.0
DEODORANTS AEROSOL DEODORANTS 3.5 OZ
suggested:  9337841 DEODORANTS AEROSOL DEODORANTS 3.5 OZ
suggested:  979613 ANALGESICS ADULT ANALGESICS  
suggested:  955888 EASTER SEASONAL MISCELLANEOUS  
suggested:  936118 FEMININE HYGIENE FEM. HYGN. TAMPONS 18CT
suggested:  12695417 LAXATIVES LAXATIVES 90 CT
suggested:  963855 BAG SNACKS SGL SV/VEN

suggested:  950850 SNACK NUTS VAC PAC NUTS(CANS) 16 OZ
suggested:  15926970 FLORAL- HARD GOODS VASES&CONTAINERS 4 INCH
suggested:  15926715 FLORAL- HARD GOODS ENHANCEMENTS & POTCOVERS 6.25 INCH
suggested:  8181567 GREETING CARDS/WRAP/PARTY SPLY SPECIAL EVERYDAY  
suggested:  987006 PWDR/CRYSTL DRNK MX SOFT DRINK CANISTERS 1.4 OZ
suggested:  10457178 SNACK NUTS SNACK NUT(BAG) 14 OZ
suggested:  1058031 BEEF CHOICE BEEF  
suggested:  9676938 FLUID MILK PRODUCTS REFRIGERATED COFFEE CREAMERS 32 OZ
suggested:  1128477 BAKED BREAD/BUNS/ROLLS PREMIUM BREAD 24 OZ
suggested:  12605049 CHRISTMAS  SEASONAL DECOR  
-------------base product:
990244.0
SMOKING CESSATIONS SMOKING CESSATION 14 CT
suggested:  990244 SMOKING CESSATIONS SMOKING CESSATION 14 CT
suggested:  8019439 PET CARE SUPPLIES RAWHIDE CHEWS 2 PK
suggested:  6773229 FROZEN PIZZA PIZZA/ECONOMY 6.1 OZ
suggested:  966070 FROZEN PIZZA PIZZA/ECONOMY 5 OZ
suggested:  13213357 ICE CREAM/MILK/SHERBTS PREMIUM PINTS 16 OZ
suggested:  12187680 LU

suggested:  6904667 ORGANICS FRUIT & VEGETABLES ORGANIC POTATOES 3 LB
suggested:  1074674 PREPARED/PKGD FOODS SAUCES 26 OZ
suggested:  12428020 GREETING CARDS/WRAP/PARTY SPLY PARTY EVERYDAY 24 COUNT
suggested:  5995078 HISPANIC MEXICAN SAUCESSALSAPICANTEE 16 OZ
suggested:  10457237 EGGS EGGS SUBSTITUTE 32 OZ
suggested:  13190461 NO COMMODITY DESCRIPTION NO SUBCOMMODITY DESCRIPTION  
suggested:  834128 DINNER SAUSAGE SAUSAGE  
suggested:  891619 BEANS - CANNED GLASS & MW VARIETY BEANS - KIDNEY PINTO  
suggested:  839840 HOT DOGS PREMIUM - MEAT 1 LB
suggested:  5564004 PEPPERS-ALL PEPPERS OTHER BELL 3 CT
-------------base product:
9296441.0
EYE AND EAR CARE PRODUCTS GENERAL EYE CARE  
suggested:  9296441 EYE AND EAR CARE PRODUCTS GENERAL EYE CARE  
suggested:  1108094 BEERS/ALES BEERALEMALT LIQUORS 12 OZ
suggested:  1105929 BEEF LOIN - STK/CHP/SLC  
suggested:  1133393 PET CARE SUPPLIES COLLARS AND LEADS 8 INCH
suggested:  1131933 DOMESTIC WINE FRUIT VARIETAL WINES 1.5 LTR
suggested:  90

suggested:  951221 CHEESE NATURAL CHEESE EXACT WT CHUNKS 8 OZ
suggested:  876948 BREAKFAST SAUSAGE/SANDWICHES OTHER 16 OZ
suggested:  1039831 DINNER MXS:DRY MACARONI & CHEESE DNRS 5.5 OZ
suggested:  1005312 CONDIMENTS/SAUCES VINEGAR ALL-EXCEPT WINE/RIC 32 OZ
suggested:  909510 DRY NOODLES/PASTA NOODLES DRY 8 OZ
suggested:  1043128 FRUIT - SHELF STABLE PINEAPPLE 20 OZ
suggested:  15452044 TEAS TEA BAGS & BULK TEA 48 CT
suggested:  991083 VEGETABLES - SHELF STABLE TOMATOES WHOLE 28 OZ
suggested:  1104702 OLIVES GREEN OLIVES 5.75 OZ
suggested:  1113142 BAG SNACKS PRETZELS 16 OZ
-------------base product:
981874.0
SALD DRSNG/SNDWCH SPRD POURABLE SALAD DRESSINGS 16 OZ
suggested:  981874 SALD DRSNG/SNDWCH SPRD POURABLE SALAD DRESSINGS 16 OZ
suggested:  9487323 BAG SNACKS POTATO CHIPS 9 OZ
suggested:  9527162 DRY NOODLES/PASTA SPAGHETTI DRY 2 LB
suggested:  985105 FRZN BREAKFAST FOODS FRZN BREAKFAST ENTREES/SANDWIC 9 OZ
suggested:  6423937 FRZN BREAKFAST FOODS FRZN BREAKFAST PASTRY 10.8 OZ
su